# Inference Pipeline

Inference Pipelines are a great way to process video streams with Inference. You can configure different sources that include streams from local devices, RTSP streams, and local video files. You can also configure different sinks that include UDP streaming of results, render of results, and custom callbacks to run your own logic after each new set of predictions is available. 

### Roboflow API Key

To load models with `inference`, you'll need a Roboflow API Key. Find instructions for retrieving your API key [here](https://docs.roboflow.com/api-reference/authentication). The utility function below attempts to load your Roboflow API key from your enviornment. If it isn't found, it then prompts you to input it. To avoid needing to input your API key for each example, you can configure your Roboflow API key in your environment via the variable `ROBOFLOW_API_KEY`.

In [ ]:
from utils import get_roboflow_api_key

api_key = get_roboflow_api_key()

### Inference Pipeline Example

In this example we create a new InferencePipeline. We pass the model ID, the video reference, and a method to render our results. Out pipeline does the rest!

In [ ]:
from functools import partial

import numpy as np
from matplotlib import pyplot as plt
from IPython import display

from inference.core.interfaces.stream.inference_pipeline import InferencePipeline
from inference.core.interfaces.stream.sinks import render_boxes

# Define source video
video_url = "https://storage.googleapis.com/com-roboflow-marketing/football-video.mp4"

# Prepare to plot results

fig, ax = plt.subplots()
frame_placeholder = np.zeros((480, 640, 3), dtype=np.uint8)  # Adjust the dimensions to match your frame size
image_display = ax.imshow(frame_placeholder)

# Define our plotting function
def update_plot(new_frame):
    # Update the image displayed
    image_display.set_data(new_frame)
    # Redraw the canvas immediately
    display.display(plt.gcf())
    display.clear_output(wait=True)

# Define our pipeline's sink
render = partial(render_boxes, on_frame_rendered=update_plot)

# Instantiate the pipeline
pipeline = InferencePipeline.init(
    model_id="soccer-players-5fuqs/1",
    video_reference=video_url,
    on_prediction=render,
)

# Start the pipeline
pipeline.start()
pipeline.join()